In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('daily_users.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
def remove_commas(x):
    return float(x.replace(',', ''))
df['Page.Loads'] = df['Page.Loads'].apply(lambda x : remove_commas(x))
df = df[['Date','Page.Loads']]
df.columns = ['ds', 'y']

In [ ]:
df.head()

In [ ]:
plt.plot(df['ds'], df['y'])
plt.show()

**Simple Moving Average**

Due to its emphasis on long-term trends and capacity to reduce the impact of short-term noise, simple moving average is a good tool for finding abnormalities. The SMA provides a smooth representation of the underlying pattern by calculating the average of data points across a chosen timeframe, which makes it simpler to spot departures from the average.

In [ ]:
df['SMA'] = df.iloc[:,1].rolling(window=30).mean()
df[['y','SMA']].plot()

In [ ]:
df['diff_SMA'] = df['y'] - df['SMA']
df['diff_SMA'].hist()
plt.title('The distribution of diff in SMA')

In [ ]:
df['upper_SMA'] = df['SMA'] + 2000
df['lower_SMA'] = df['SMA'] - 2000
df[30:60]

In [ ]:
def plot_it():
    plt.plot(df['y'],'go',markersize=2,label='Actual')
    plt.fill_between(
       np.arange(df.shape[0]), df['lower_SMA'], df['upper_SMA'], alpha=0.5, color="r",
       label="Predicted interval")
    plt.xlabel("Ordered samples.")
    plt.ylabel("Values and prediction intervals.")
    plt.show()
    
plot_it()

**Exponential Smoothing**

Exponential smoothing is particularly sensitive to abrupt shifts or odd patterns in the time series because it can give higher weights to recent data points. Due to its sensitivity, it may efficiently detect and highlight anomalies in the most recent observations.

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing
EMAfit = SimpleExpSmoothing(df['y']).fit(smoothing_level=0.2,optimized=False)
EMA = EMAfit.forecast(3).rename(r'$\alpha=0.2$')
df['EMA'] = EMAfit.predict(start = 0)
df[['y','EMA']].plot()

In [ ]:
df['diff_EMA'] = df['y'] - df['EMA']
df['diff_EMA'].hist()
plt.title('The distribution of diff in EMA')

In [ ]:
df['upper_EMA'] = df['EMA'] + 2000
df['lower_EMA'] = df['EMA'] - 2000
df[30:60]

In [ ]:
def plot_it():
    plt.plot(df['y'],'go',markersize=2,label='Actual')
    plt.fill_between(
       np.arange(df.shape[0]), df['lower_EMA'], df['upper_EMA'], alpha=0.5, color="r",
       label="Predicted interval")
    plt.xlabel("Ordered samples.")
    plt.ylabel("Values and prediction intervals.")
    plt.show()
    
plot_it()

**Seasonal-Trend Decomposition (STD)**

Time series anomalies are divided into their seasonal, trend, and residual components using the Seasonal-Trend Decomposition (STD) method. Using STD, we can efficiently separate the seasonal and trend components, allowing for a more in-depth analysis of the residual component. The variances from the anticipated seasonal and trend patterns are represented by the residual component.

In [ ]:
import statsmodels.api as sm

df = pd.read_csv('daily_users.csv')
df['Date'] = pd.to_datetime(df['Date'])
def remove_commas(x):
    return float(x.replace(',', ''))
df['Page.Loads'] = df['Page.Loads'].apply(lambda x : remove_commas(x))
df = df[['Date','Page.Loads']]
df.columns = ['ds', 'y']
df = df.reset_index(drop='index')
df.index = pd.to_datetime(df['ds'])

result = sm.tsa.seasonal_decompose(df['y'], model='additive')
result.trend[1:2000].plot()

In [ ]:
result.seasonal[1:2000].plot()

In [ ]:
result.resid[1:2000].plot()

**Prophet**

Prophet is unique in its capacity to take into consideration the impacts of seasonality, trends, and holidays on time series data. Prophet uses a Bayesian technique that effectively handles gaps in the dataset, including missing values, in contrast to simple moving average and seasonal-trend decomposition. In order to accurately capture and detect abnormalities or irregularities in the data, it establishes the uncertainty interval around the predicted values.

In [ ]:
from prophet import Prophet
import plotly.offline as py

py.init_notebook_mode()
%matplotlib inline

# Fitting with default parameters
df_model_0 = Prophet(daily_seasonality=True)
df_model_0.fit(df)

In [ ]:
future= df_model_0.make_future_dataframe(periods=30, freq='d')
future.tail()

In [ ]:
df_model_0_data=df_model_0.predict(future)
df_model_0_data.tail()

In [ ]:
from prophet.plot import add_changepoints_to_plot

fig= df_model_0.plot(df_model_0_data)
a = add_changepoints_to_plot(fig.gca(), df_model_0, df_model_0_data)

In [ ]:
df_model_0.plot_components(df_model_0_data)